In [1]:
!unzip /content/vectorstore.zip -d /content/

Archive:  /content/vectorstore.zip
   creating: /content/vectorstore/
   creating: /content/vectorstore/9dd227e7-72e3-4646-a74b-43c7d511e8fd/
  inflating: /content/vectorstore/9dd227e7-72e3-4646-a74b-43c7d511e8fd/data_level0.bin  
  inflating: /content/vectorstore/9dd227e7-72e3-4646-a74b-43c7d511e8fd/header.bin  
  inflating: /content/vectorstore/9dd227e7-72e3-4646-a74b-43c7d511e8fd/length.bin  
  inflating: /content/vectorstore/9dd227e7-72e3-4646-a74b-43c7d511e8fd/link_lists.bin  
  inflating: /content/vectorstore/chroma.sqlite3  


In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl", trust_remote_code=True)
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xl")

generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)

prompt = "Hello, what is the capital of Pakistam"
result = generator(prompt, max_new_tokens=150, do_sample=False)

print(result[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


Karachi


In [2]:
!pip install -U langchain-community

In [10]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 101.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 114.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 7.1 MB/s eta 

In [20]:

from langchain.prompts import PromptTemplate
from transformers import pipeline
import json
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
import os

embedding_model = SentenceTransformerEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


def get_relevant_docs_basic(user_query):
    vectordb = Chroma(persist_directory="/content/vectorstore",
                      embedding_function=embedding_model)
    retriever = vectordb.as_retriever(score_threshold=0.5)
    relevant_docs = retriever.invoke(user_query)
    return relevant_docs




def make_rag_prompt(query, relevant_passage):
    task_guidance = """
    You are an intelligent, formal, and trustworthy AI assistant for a local bank.
    Use the retrieved context below to answer the customer's question accurately using the context word to word.
    Do not repeat the question or include any extra information, explanations, or reasoning.
    Just provide the direct answer in English, along with the source of the information.

    If the answer is not present in the context, respond with: "I'm sorry, I do not have that information at the moment."
"""

    template = PromptTemplate(
        input_variables=["query", "relevant_passage"],
        template=(
            "{task_guidance}\n\n"
            "Context:\n{relevant_passage}\n\n"
            "Question: {query}\n"
            "Answer (include source):"
        )
    )

    return template.format(
        query=query,
        relevant_passage=relevant_passage,
        task_guidance=task_guidance.strip()
    )


def generate_response(user_prompt):
    generator = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0)
    result = generator(user_prompt, max_new_tokens=1000, do_sample=False)
    return (result[0]["generated_text"])


def generate_answer(query):
    relevant_text = get_relevant_docs_basic(query)
    # text = " \n".join([doc.page_content for doc in relevant_text])
    prompt = make_rag_prompt(query, relevant_passage=relevant_text)
    # print(prompt)
    # print(prompt)
    answer = generate_response(prompt)
    return answer

In [21]:
answer = generate_answer("What is the opening and minimum balance requirement in NAA?.")
print(answer)

Device set to use cuda:0


account can be opened with rs.100 and there is no minimum balance requirement. nust asaan account latest rate sheet


In [26]:
answer = generate_answer("In which area of the country one can avail Personal Finance form NUST")
print(answer)

Device set to use cuda:0
Token indices sequence length is longer than the specified maximum sequence length for this model (535 > 512). Running this sequence through the model will result in indexing errors


all cities where nust branches exists
